#S26 - Qué porcentaje de las versiones actuales son páginas que se editaron una sola vez (⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=e5e852ace02c5287d039101dda1ec8e7ecb9a740684ca524a39ca684d8043102
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/drive/MyDrive/Datasets/contents.csv')

In [ ]:
contents_rdd = df.rdd

In [ ]:
contents_rdd

MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
contents_rdd.getNumPartitions()

5

In [ ]:
contents_rdd.take(20)

[Row(title='Dade City', id='270173', namespace='0', revision_id='119487500', parent_revision='107421575', revision_timestamp='2019-09-18T12:40:05Z', revisor_username='Aosbot', revisor_id='5045504', revisor_ip=None, revisor_comment='Mantenimiento de Control de autoridades'),
 Row(title='Categoría:Tratados de 1985', id='7140595', namespace='14', revision_id='91948211', parent_revision='86887108', revision_timestamp='2016-06-27T16:38:36Z', revisor_username='Sabbut', revisor_id='2075', revisor_ip=None, revisor_comment=None),
 Row(title='Manzanilla de oro', id='3130871', namespace='0', revision_id='31988275', parent_revision=None, revision_timestamp='2009-12-05T11:42:06Z', revisor_username='MILEPRI', revisor_id='319121', revisor_ip=None, revisor_comment='Redirigiendo a «[[Chrysanthellum americanum]]»'),
 Row(title='Luis Mantilla Guerrero', id='8841925', namespace='0', revision_id='129809090', parent_revision='125715288', revision_timestamp='2020-10-05T10:38:20Z', revisor_username='Born2bgra

Cuento con 1 si fue editado solo una vez (es decir no tiene parent revision) y con 0 si fue editado más de una vez (es decir tiene parent revision). A esta tupla le agrego un 1 para contabilizar los totales.
Luego hago un reduce que devuelva una tupla, por un lado los que fueron editados una vez, y por otro los totales, para finalmente sacar el porcentaje.

In [ ]:
un_edit, totales = contents_rdd.map(lambda x: (1 if x[4] == None else 0, 1)).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
(un_edit / totales) * 100

35.90088933616897